In [1]:
import os
import sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [2]:
start_pos_numeric_name = {
    (0, 0): 1,
    (0, 1): 2,
    (0, 2): 3,
    (0, 3): 4,
    (0, 4): 5,
    (0, 5): 6,
    (0, 6): 7,
    (0, 7): 8,
    (1, 0): 9,
    (1, 1): 10,
    (1, 2): 11,
    (1, 3): 12,
    (1, 4): 13,
    (1, 5): 14,
    (1, 6): 15,
    (1, 7): 16,
    (6, 0): -16,
    (6, 1): -15,
    (6, 2): -14,
    (6, 3): -13,
    (6, 4): -12,
    (6, 5): -11,
    (6, 6): -10,
    (6, 7): -9,
    (7, 0): -8,
    (7, 1): -7,
    (7, 2): -6,
    (7, 3): -5,
    (7, 4): -4,
    (7, 5): -3,
    (7, 6): -2,
    (7, 7): -1
}

In [3]:
!ls -alrth /data_data/reinforcement_learning/results/comb*

-rw-rw-r-- 1 ubuntu ubuntu 1.4G Jun 28 08:39 /data_data/reinforcement_learning/results/combined_file.bak.gz
-rw-rw-r-- 1 ubuntu ubuntu  69G Jun 28 08:47 /data_data/reinforcement_learning/results/combined_policy_file
-rw-rw-r-- 1 ubuntu ubuntu  25G Jun 28 08:54 /data_data/reinforcement_learning/results/combined_policy_file.bak
-rw-rw-r-- 1 ubuntu ubuntu  59G Jun 28 09:01 /data_data/reinforcement_learning/results/combined_file
-rw-rw-r-- 1 ubuntu ubuntu 1.5G Jun 28 09:03 /data_data/reinforcement_learning/results/combined_policy_file.bak.gz
-rw-rw-r-- 1 ubuntu ubuntu 1.7G Jun 28 09:06 /data_data/reinforcement_learning/results/combined_file.gz
-rw-rw-r-- 1 ubuntu ubuntu 469M Aug 13 00:33 /data_data/reinforcement_learning/results/combined_policy_file_1k
-rw-rw-r-- 1 ubuntu ubuntu 463M Aug 13 00:34 /data_data/reinforcement_learning/results/combined_policy_file_1M


In [4]:
!head -n 10240000 /data_data/reinforcement_learning/results/combined_policy_file >> /data_data/reinforcement_learning/results/combined_policy_file_10M

In [1]:
!head -n 10 /data_data/reinforcement_learning/results/combined_policy_file_10M

{"state_prior":"000000000000000000000000000000000000000000000000000000000011011100000110000000011111000000000000000001000000001100000000001101000000100001100000000000000000000000000101000000000000011110000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010010000000010011010100000000010101011000011001","expected_value":4.166666666666667,"policy":"(Pawn,(1,0),(1,2),(2,2))"}
{"state_prior":"000000000000000000000000000000000000000000000000000000000011011100000110000000011111000000000000000001000000001100000000001101000000100001100000000000000000000000000101000000000000011110000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010010000000010011010100000000010101011000011001","expected_value":0.2331338002025748,"policy":"(Pawn,(1,0),(1,2),(2,2))"}
{"state_prior":"00000000000000000000000000000000000

In [5]:
src_file = '/data_data/reinforcement_learning/results/combined_policy_file_10M'

In [6]:
train_pctl       = 0.65
validation_pctl  = 0.80
test_pctl        = 0.20


In [7]:
state  = None
value  = None
action = None

### Refactor as Spark job...

In [ ]:
from collections import defaultdict
import ast

from itertools import cycle, compress

from sparse_action_dict import sparse_action_dict

partition = {}
labels    = {}

partition['train']      = []
partition['validation'] = []

partitions = ['train','validation']

with open(src_file,'r') as policy_file:
    policies_        = policy_file.readlines()
    num_policies     = len(policies_)
    train_cut_pt     = int(train_pctl*num_policies)
    val_cut_pt       = int(validation_pctl*num_policies)
    policy_num       = 0
    
    for part in partitions:
        if part      == 'train':
            policies = policies_[:train_cut_pt]
        if part      == 'validation':
            policies = policies_[train_cut_pt:val_cut_pt]
    
        for policy_ in policies:
            policy   = ast.literal_eval(policy_)
            state, value, action = policy.values()
            state    = np.array(list(map(int, state))).astype(np.float32)
            action   = [int(x) for x in action.replace("(","").replace(")","").split(",")[1:]]
            
            start_pos, curr_pos, next_pos   = list(map(list, zip(compress(action, cycle([1, 0])), compress(action, cycle([0, 1])))))
            
            move0 = tuple(next_pos)[0] - tuple(curr_pos)[0]
            move1 = tuple(next_pos)[1] - tuple(curr_pos)[1]
            
            try:
                sparse_move = str(start_pos_numeric_name[tuple(start_pos)]) + "," + str(move0) + "," + str(move1)
            
                action_ = np.array(list(map(int, sparse_action_dict[sparse_move]))).astype(np.float32)


                state_action = str(np.concatenate((state,action_), axis=0))


                if not (state_action) in partition[part]:
                    partition[part].append(state_action)
                    labels.update({state_action:value})
                else:
                    curr_policy_value = labels[state_action]
                    if value > curr_policy_value:
                        labels.update({state_action:value})
                    else:
                        pass
            except:
                #Pass for now -- but a big issue here....
                pass
                
            policy_num += 1
            

In [ ]:
partition_ = {}
partition_['train']      = []
partition_['validation'] = []
parts = ['train','validation']

for part in parts:

    for state_ in partition[part]:
        state = np.array([int(x) for x in np.array(state_.strip().replace("\n","").replace(".","").replace("''","").replace("]","").replace("[","").split(" "))])
        partition_[part].append(state)

In [ ]:
labels_ = {}

for state_action_, value in labels.items():
    state_action = str([int(x) for x in np.array(state_action_.strip().replace("\n","").replace(".","").replace("''","").replace("]","").replace("[","").split(" "))])
    labels_.update({state_action:value})

In [ ]:
num_policies     = len(policies_)
train_cut_pt     = int(train_pctl*num_policies)
val_cut_pt       = int(validation_pctl*num_policies)

### Dataset
From https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

In [ ]:
%%writefile my_classes.py

import torch

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels   = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.tensor( ID + '.pt')
        y = self.labels[ID]

        return X, y

From https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

Now, we have to modify our PyTorch script accordingly so that it accepts the generator that we just created. In order to do so, we use PyTorch's DataLoader class, which in addition to our Dataset class, also takes in the following important arguments:

- batch_size:  Denotes the number of samples contained in each generated batch.
- shuffle:     If set to True, we will get a new order of exploration at each pass (or just keep a linear exploration scheme otherwise). Shuffling the order in which examples are fed to the classifier is helpful so that batches between epochs do not look alike. Doing so will eventually make our model more robust.
- num_workers: Denotes the number of processes that generate batches in parallel. A high enough number of workers assures that CPU computations are efficiently managed, i.e. that the bottleneck is indeed the neural network's forward and backward operations on the GPU (and not data generation).

A proposition of code template that you can write in your script is shown below.

In [ ]:
from agent import Agent
from model import MCritic
from replaybuffer import ReplayBuffer
from args import args

mCritic       =  MCritic(396)

In [ ]:
print(mCritic.network)

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from my_classes import Dataset
from random import sample
from operator import itemgetter

sys.path.append('./anaconda3/lib/python3.7/site-packages/torchvision')
from tensorboardX import SummaryWriter
writer = SummaryWriter(comment="MADDPG Chessy II")

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cuda:1")
#torch.backends.cudnn.benchmark = False

# Parameters
params = {'batch_size': 1024,
          'shuffle': True,
          'num_workers': 2}

max_epochs = 100000

num_policies       = len(labels.keys())
train_cut_pt       = int(train_pctl*num_policies)
val_cut_pt         = int(validation_pctl*num_policies)
training_generator = None

def load_data():
    
    global training_generator
    missing_train_samples = 0
    missing_val_samples = 0
    partition      = partition_
    labels         = labels_
    sample_train   = list(np.arange(0,len(partition['train']),1))
    sample_val     = list(np.arange(0,len(partition['validation']),1))

    for epoch in range(max_epochs):
        # Training
        train_idxs   = sample(sample_train, params['batch_size'])
            
        train_data   = torch.tensor([x for x in map(partition['train'].__getitem__, (tuple(train_idxs)))]).float().to(device)
            
        train_labels = torch.tensor([labels_[str(list(partition_['train'][x]))] for x in train_idxs]).float().reshape((params['batch_size'],)).to(device)
                    
        local_batch  = train_data.to(device)
        
        local_labels = train_labels.view(params['batch_size'], -1).to(device)
        
        Q_targets    = local_labels

        Q_expected   = mCritic.network(local_batch)
        
        mCritic_loss = F.mse_loss(Q_expected, Q_targets)

        mCriticLoss  = mCritic_loss

        
        # Minimize the loss
        mCritic.optimizer.zero_grad()
        mCritic_loss.backward()
        mCritic.optimizer.step()
            
        # Validation
        with torch.set_grad_enabled(False):
            
            val_idxs     = sample(sample_val, params['batch_size'])
        
            val_data     = torch.tensor([x for x in map(partition['validation'].__getitem__, (tuple(val_idxs)))]).float().to(device)

            val_labels   = torch.tensor([labels_[str(list(partition_['validation'][x]))] for x in val_idxs]).float().reshape((params['batch_size'],)).to(device)

            val_batch    = val_data.to(device) 
            
            val_labels   = val_labels.view(params['batch_size'], -1).to(device)
            
            V_targets    = val_labels
            
            V_expected   = mCritic.network(val_batch)
            
            mCritic_val_loss = F.mse_loss(V_expected, V_targets)

            
        if epoch % 100 == 0 and epoch > 0:
            num_iter   = epoch
            writer.add_scalars('mCritic_loss_II', {
                'mCritic_train_loss': mCritic_loss, 
                'mCritic_val_loss': mCritic_val_loss,
            },  num_iter)
            print("Epoch:\t{}\tmCritic_train_loss:\t{}\tmCritic_val_loss:\t{}\n".format(epoch,mCritic_loss, mCritic_val_loss))
            
            
            

In [ ]:
load_data()

In [ ]:
feasible_moves = [('Pawn',(1,0),(1,2),(2,2))]

In [ ]:
numeric_names = {'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1':
 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3':
 -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1'
: -3, 'b_K1': -2, 'b_R1': -1}

In [ ]:
%%writefile args.py

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


device  = torch.device("cuda:0" if torch.cuda.is_available() else "cuda:1")
device

args = { 
    "update_every":100,
    "window_size":100,
    "BUFFER_SIZE":int(1e6),
    "BATCH_SIZE":1024,  
    "GAMMA":0.99,
    "TAU":2e-3,
    "LR_ACTOR":1e-3,
    "LR_CRITIC":1.1e-3,
    "WEIGHT_DECAY":0.0001,
    "UPDATE_EVERY":5,
    "EXPLORE_NOISE":0.05,
    "FC1_UNITS":768,
    "FC2_UNITS":2048,
    "FC3_UNITS":256,
    "seed":0,
    "state_size":384,
    "action_size":88,
    "action_size_binary":12,
    "num_agents":2,
    "device":torch.device("cuda:0" if torch.cuda.is_available() else "cuda:1"),
    'mcritic_path':'/home/ubuntu/chessy/checkpoint_mCritic.pth',
    'agent_p0_path':'/home/ubuntu/chessy/checkpoint_p0.pth',
    'agent_p1_path':'/home/ubuntu/chessy/checkpoint_p1.pth',
    'action_id_dict': {'1,1,0': 0, '2,1,-2': 1, '2,1,2': 2, '2,2,-1': 3, '2,2,1': 4, '3,1,-1': 5, '3,1,1': 6, '4,1,-1': 7, '4,1,0': 8, '4,1,1': 9, '5,1,-1': 10, '5,1,0': 11, '5,1,1': 12, '6,1,-1': 13, '6,1,1': 14, '7,1,-2': 15, '7,1,2': 16, '7,2,-1': 17, '7,2,1': 18, '8,1,0': 19, '9,1,0': 20, '9,1,-1': 21, '9,1,1': 22, '10,1,0': 23, '10,1,-1': 24, '10,1,1': 25, '11,1,0': 26, '11,1,-1': 27, '11,1,1': 28, '12,1,0': 29, '12,1,-1': 30, '12,1,1': 31, '13,1,0': 32, '13,1,-1': 33, '13,1,1': 34, '14,1,0': 35, '14,1,-1': 36, '14,1,1': 37, '15,1,0': 38, '15,1,-1': 39, '15,1,1': 40, '16,1,0': 41, '16,1,-1': 42, '16,1,1': 43, '-16,-1,0': 44, '-16,-1,-1': 45, '-16,-1,1': 46, '-15,-1,0': 47, '-15,-1,-1': 48, '-15,-1,1': 49, '-14,-1,0': 50, '-14,-1,-1': 51, '-14,-1,1': 52, '-13,-1,0': 53, '-13,-1,-1': 54, '-13,-1,1': 55, '-12,-1,0': 56, '-12,-1,-1': 57, '-12,-1,1': 58, '-11,-1,0': 59, '-11,-1,-1': 60, '-11,-1,1': 61, '-10,-1,0': 62, '-10,-1,-1': 63, '-10,-1,1': 64, '-9,-1,0': 65, '-9,-1,-1': 66, '-9,-1,1': 67, '-8,-1,0': 68, '-7,-1,-2': 69, '-7,-1,2': 70, '-7,-2,-1': 71, '-7,-2,1': 72, '-6,-1,-1': 73, '-6,-1,1': 74, '-5,-1,-1': 75, '-5,-1,0': 76, '-5,-1,1': 77, '-4,-1,-1': 78, '-4,-1,0': 79, '-4,-1,1': 80, '-3,-1,-1': 81, '-3,-1,1': 82, '-2,-1,-2': 83, '-2,-1,2': 84, '-2,-2,-1': 85, '-2,-2,1': 86, '-1,-1,0': 87},
    'id_action_dict': {0: '1,1,0', 1: '2,1,-2', 2: '2,1,2', 3: '2,2,-1', 4: '2,2,1', 5: '3,1,-1', 6: '3,1,1', 7: '4,1,-1', 8: '4,1,0', 9: '4,1,1', 10: '5,1,-1', 11: '5,1,0', 12: '5,1,1', 13: '6,1,-1', 14: '6,1,1', 15: '7,1,-2', 16: '7,1,2', 17: '7,2,-1', 18: '7,2,1', 19: '8,1,0', 20: '9,1,0', 21: '9,1,-1', 22: '9,1,1', 23: '10,1,0', 24: '10,1,-1', 25: '10,1,1', 26: '11,1,0', 27: '11,1,-1', 28: '11,1,1', 29: '12,1,0', 30: '12,1,-1', 31: '12,1,1', 32: '13,1,0', 33: '13,1,-1', 34: '13,1,1', 35: '14,1,0', 36: '14,1,-1', 37: '14,1,1', 38: '15,1,0', 39: '15,1,-1', 40: '15,1,1', 41: '16,1,0', 42: '16,1,-1', 43: '16,1,1', 44: '-16,-1,0', 45: '-16,-1,-1', 46: '-16,-1,1', 47: '-15,-1,0', 48: '-15,-1,-1', 49: '-15,-1,1', 50: '-14,-1,0', 51: '-14,-1,-1', 52: '-14,-1,1', 53: '-13,-1,0', 54: '-13,-1,-1', 55: '-13,-1,1', 56: '-12,-1,0', 57: '-12,-1,-1', 58: '-12,-1,1', 59: '-11,-1,0', 60: '-11,-1,-1', 61: '-11,-1,1', 62: '-10,-1,0', 63: '-10,-1,-1', 64: '-10,-1,1', 65: '-9,-1,0', 66: '-9,-1,-1', 67: '-9,-1,1', 68: '-8,-1,0', 69: '-7,-1,-2', 70: '-7,-1,2', 71: '-7,-2,-1', 72: '-7,-2,1', 73: '-6,-1,-1', 74: '-6,-1,1', 75: '-5,-1,-1', 76: '-5,-1,0', 77: '-5,-1,1', 78: '-4,-1,-1', 79: '-4,-1,0', 80: '-4,-1,1', 81: '-3,-1,-1', 82: '-3,-1,1', 83: '-2,-1,-2', 84: '-2,-1,2', 85: '-2,-2,-1', 86: '-2,-2,1', 87: '-1,-1,0'},
    'numeric_names' : {'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1': 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3': -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1': -3, 'b_K1': -2, 'b_R1': -1},
    'initial_state'  : "010010010011010100010101010110010111011000011001011010011011011100011101011110011111100000100001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000011000100000101000110000111001000001001001010001011001100001101001110001111010000",
    'SUMMARY_FILE':"/data_data/reinforcement_learning/results/summary_file.tsv",
    'HISTORY_FILE':None,
    'TAKE_KING_REWARD':10,
    'MORE_POINTS_REWARD':1,
    'EQUAL_POINTS_REWARD':0,
    'STEP_REWARD':-1,
    'WINS_DRAWS_LOSSES': [0,0,0],
    'in_channels_1':1024,
    'in_channels_2':64,
    'in_channels_l':128,
    'out_channels_1':64,
    'out_channels_2':128,
    'out_channels_l':16,
    'kernel_1_size':(6,1),
    'kernel_2_size':(1,1),
    'kernel_l_size':(1,1),
    'stride_1_size':(6,1),
    'stride_2_size':(6,1),
    'stride_l_size':(1,1),
    'reshape_size':(8,48),
    'reshape_buffer':(1,384)
}

In [ ]:
action_id_dict = {'1,1,0': 0, '2,1,-2': 1, '2,1,2': 2, '2,2,-1': 3, '2,2,1': 4, '3,1,-1': 5, '3,1,1': 6, '4,1,-1': 7, '4,1,0': 8, '4,1,1': 9, '5,1,-1': 10, '5,1,0': 11, '5,1,1': 12, '6,1,-1': 13, '6,1,1': 14, '7,1,-2': 15, '7,1,2': 16, '7,2,-1': 17, '7,2,1': 18, '8,1,0': 19, '9,1,0': 20, '9,1,-1': 21, '9,1,1': 22, '10,1,0': 23, '10,1,-1': 24, '10,1,1': 25, '11,1,0': 26, '11,1,-1': 27, '11,1,1': 28, '12,1,0': 29, '12,1,-1': 30, '12,1,1': 31, '13,1,0': 32, '13,1,-1': 33, '13,1,1': 34, '14,1,0': 35, '14,1,-1': 36, '14,1,1': 37, '15,1,0': 38, '15,1,-1': 39, '15,1,1': 40, '16,1,0': 41, '16,1,-1': 42, '16,1,1': 43, '-16,-1,0': 44, '-16,-1,-1': 45, '-16,-1,1': 46, '-15,-1,0': 47, '-15,-1,-1': 48, '-15,-1,1': 49, '-14,-1,0': 50, '-14,-1,-1': 51, '-14,-1,1': 52, '-13,-1,0': 53, '-13,-1,-1': 54, '-13,-1,1': 55, '-12,-1,0': 56, '-12,-1,-1': 57, '-12,-1,1': 58, '-11,-1,0': 59, '-11,-1,-1': 60, '-11,-1,1': 61, '-10,-1,0': 62, '-10,-1,-1': 63, '-10,-1,1': 64, '-9,-1,0': 65, '-9,-1,-1': 66, '-9,-1,1': 67, '-8,-1,0': 68, '-7,-1,-2': 69, '-7,-1,2': 70, '-7,-2,-1': 71, '-7,-2,1': 72, '-6,-1,-1': 73, '-6,-1,1': 74, '-5,-1,-1': 75, '-5,-1,0': 76, '-5,-1,1': 77, '-4,-1,-1': 78, '-4,-1,0': 79, '-4,-1,1': 80, '-3,-1,-1': 81, '-3,-1,1': 82, '-2,-1,-2': 83, '-2,-1,2': 84, '-2,-2,-1': 85, '-2,-2,1': 86, '-1,-1,0': 87}

id_action_dict = {0: '1,1,0', 1: '2,1,-2', 2: '2,1,2', 3: '2,2,-1', 4: '2,2,1', 5: '3,1,-1', 6: '3,1,1', 7: '4,1,-1', 8: '4,1,0', 9: '4,1,1', 10: '5,1,-1', 11: '5,1,0', 12: '5,1,1', 13: '6,1,-1', 14: '6,1,1', 15: '7,1,-2', 16: '7,1,2', 17: '7,2,-1', 18: '7,2,1', 19: '8,1,0', 20: '9,1,0', 21: '9,1,-1', 22: '9,1,1', 23: '10,1,0', 24: '10,1,-1', 25: '10,1,1', 26: '11,1,0', 27: '11,1,-1', 28: '11,1,1', 29: '12,1,0', 30: '12,1,-1', 31: '12,1,1', 32: '13,1,0', 33: '13,1,-1', 34: '13,1,1', 35: '14,1,0', 36: '14,1,-1', 37: '14,1,1', 38: '15,1,0', 39: '15,1,-1', 40: '15,1,1', 41: '16,1,0', 42: '16,1,-1', 43: '16,1,1', 44: '-16,-1,0', 45: '-16,-1,-1', 46: '-16,-1,1', 47: '-15,-1,0', 48: '-15,-1,-1', 49: '-15,-1,1', 50: '-14,-1,0', 51: '-14,-1,-1', 52: '-14,-1,1', 53: '-13,-1,0', 54: '-13,-1,-1', 55: '-13,-1,1', 56: '-12,-1,0', 57: '-12,-1,-1', 58: '-12,-1,1', 59: '-11,-1,0', 60: '-11,-1,-1', 61: '-11,-1,1', 62: '-10,-1,0', 63: '-10,-1,-1', 64: '-10,-1,1', 65: '-9,-1,0', 66: '-9,-1,-1', 67: '-9,-1,1', 68: '-8,-1,0', 69: '-7,-1,-2', 70: '-7,-1,2', 71: '-7,-2,-1', 72: '-7,-2,1', 73: '-6,-1,-1', 74: '-6,-1,1', 75: '-5,-1,-1', 76: '-5,-1,0', 77: '-5,-1,1', 78: '-4,-1,-1', 79: '-4,-1,0', 80: '-4,-1,1', 81: '-3,-1,-1', 82: '-3,-1,1', 83: '-2,-1,-2', 84: '-2,-1,2', 85: '-2,-2,-1', 86: '-2,-2,1', 87: '-1,-1,0'}
    

In [ ]:
env = None

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


device  = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:0")
device

In [ ]:
from games  import games
from game   import Game
from player import Player_Template
from team   import Team
from userinput import userInput

def run_trials(user_input):
    """ Runs the num_trials """
    global cycle
    global HISTORY_FILE
    global SUMMARY_FILE
    global env
    global agent_scores
    
    print("==============================RUN_TRIALS==================================")
    

    num_trials = user_input.num_trials
    num_sides  = user_input.num_sides
        
    env = Game(user_input.game, 8, num_sides, user_input.display_board_positions)


In [ ]:
user_input = userInput()

In [ ]:
run_trials(user_input)

In [ ]:
for j in range(user_input.num_sides):
    team_name = "team_" + str(j)
    team_name = Team(user_input.game, user_input.teams[j]["team_name"], j,user_input.teams[j]["skill"] / 10, user_input.teams[j]["strategy"])

    env.insert_team(team_name)

In [ ]:
start_pos_numeric_name = {}

In [ ]:
start_pos_numeric_name_ = {start_pos_numeric_name.update({k:numeric_names[v]}) if v else None for k,v in env.board.items()}
start_pos_numeric_name

In [ ]:
472 - 384